To do:
1. double check logstic convergence issues
2. check alpha range in supplementary repo

In [1]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from rca import make_binary_scorer, make_multiclass_scorer, process_categorical, best_logistic_solver, checker, k_fold_cross_val
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Loading Data

In [9]:
# Pulling and standardising embeddings
embeds = {}
embeds_path = '../../data/raw/embeds/'
for f_name in tqdm(os.listdir(embeds_path)):
    if f_name != 'feature_overlap.csv':  # dropping since contains many NaNs
        embed = pd.read_csv(embeds_path + f_name, index_col=0)
        embed_name = f_name.split('.')[0]
        embeds[embed_name] = (embed - embed.mean()) / embed.std()

{name: embed.shape for name, embed in embeds.items()}

  0%|          | 0/26 [00:00<?, ?it/s]

{'fMRI_text_hyper_align': (1205, 1000),
 'norms_sensorimotor': (36854, 11),
 'EEG_text': (3355, 104),
 'LexVec_CommonCrawl': (87635, 300),
 'fastText_CommonCrawl': (88986, 300),
 'spherical_text_Wikipedia': (59012, 300),
 'GloVe_CommonCrawl': (88440, 300),
 'EEG_speech': (1591, 130),
 'PPMI_SVD_SOUTH_FLORIDA': (4959, 300),
 'THINGS': (1562, 49),
 'fMRI_speech_hyper_align': (579, 6),
 'CBOW_GoogleNews': (79279, 300),
 'compo_attribs': (534, 62),
 'morphoNLM': (50506, 50),
 'microarray': (626, 15),
 'fastText_Wiki_News': (81728, 300),
 'fastTextSub_OpenSub': (72538, 300),
 'SGSoftMaxOutput_SWOW': (25442, 300),
 'PPMI_SVD_SWOW': (11783, 300),
 'GloVe_Twitter': (48614, 200),
 'GloVe_Wikipedia': (68943, 300),
 'eye_tracking': (7486, 6),
 'SGSoftMaxInput_SWOW': (11783, 300),
 'SVD_sim_rel': (6002, 300),
 'PPMI_SVD_EAT': (7775, 300)}

In [12]:
norms = pd.read_csv('../../data/raw/psychNorms.zip', index_col=0, compression='zip', low_memory=False)
norm_metadata = pd.read_csv('../../data/raw/psychNorms_metadata.csv', index_col='norm')
norm_metadata['associated_embed'] = norm_metadata['associated_embed'].astype(str)
norms

,Freq_HAL,Freq_KF,Freq_SUBTLEXUS,Freq_SUBTLEXUK,Freq_Blog,Freq_Twitter,Freq_News,Freq_CobW,Freq_CobS,CD_SUBTLEXUS,...,reproduction_vanarsdall,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall,familiarity_fear,aoa_fear,imageability_fear
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.62,4.38,2.93
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.79,3.07,5.29
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.29,3.38,5.64
March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.43,2.76,3.50


In [13]:
with open('../../data/raw/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_to_type

{'CBOW_GoogleNews': 'text',
 'fastText_CommonCrawl': 'text',
 'fastText_Wiki_News': 'text',
 'fastTextSub_OpenSub': 'text',
 'GloVe_CommonCrawl': 'text',
 'GloVe_Twitter': 'text',
 'GloVe_Wikipedia': 'text',
 'LexVec_CommonCrawl': 'text',
 'morphoNLM': 'text',
 'spherical_text_Wikipedia': 'text',
 'eye_tracking': 'brain',
 'EEG_speech': 'brain',
 'EEG_text': 'brain',
 'fMRI_speech_hyper_align': 'brain',
 'fMRI_text_hyper_align': 'brain',
 'microarray': 'brain',
 'PPMI_SVD_SWOW': 'behavior',
 'SGSoftMaxInput_SWOW': 'behavior',
 'SGSoftMaxOutput_SWOW': 'behavior',
 'PPMI_SVD_SOUTH_FLORIDA': 'behavior',
 'THINGS': 'behavior',
 'feature_overlap': 'behavior',
 'norms_sensorimotor': 'behavior',
 'compo_attribs': 'behavior',
 'SVD_sim_rel': 'behavior'}

## Cross Validation

In [14]:
# Ridge
alphas = np.logspace(-3, 3, 6)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scorer = make_binary_scorer()
multiclass_scorer = make_multiclass_scorer()

# outer_cv setting 
outer_cv, n_jobs = 5, 8

In [ ]:
# RCA
rca = []
for embed_name in tqdm(embeds.keys()):
    embed = embeds[embed_name]
    
    to_print = []
    for norm_name in tqdm(norms.columns, desc=embed_name):
        
        # Aligning data
        y = norms[norm_name].dropna()
        X, y = embed.align(y, axis=0, join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = norm_metadata.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators
        if norm_dtype in ['binary', 'multiclass']:
            X, y = process_categorical(X, y, outer_cv, inner_cv)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(X, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv),
                solver=solver, n_jobs=8
            )
            scoring = binary_scorer if norm_dtype == 'binary' else multiclass_scorer
        else: # continuous
            estimator, scoring = ridge, 'r2'

            
        # Cross validation
        check = checker(
            embed_name, y, norm_dtype, norm_metadata, outer_cv, norm_name
        )
        if check == 'pass':
             scores = k_fold_cross_val(estimator, X, y, outer_cv, scoring, n_jobs)
             r2_mean, r2_sd = scores.mean(), scores.std()
        else:
            r2_mean, r2_sd = np.nan, np.nan
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(X))
        p = X.shape[1]
        embed_type = embed_to_type[embed_name]
        rca.append([embed_name, embed_type, norm_name, train_n, p, r2_mean, r2_sd, check])
        
        to_print.append([norm_name, train_n, r2_mean, r2_sd, check])

    to_print = (
        pd.DataFrame(to_print, columns=['norm' , 'train_n', 'r2_mean', 'r2_sd', 'check'])
        .sort_values('r2_mean', ascending=False).head(10)
    )
    print(to_print)

rca = pd.DataFrame(rca, columns=['embed', 'embed_type', 'norm', 'train_n', 'p', 'r2_mean', 'r2_sd', 'check'])
rca.to_csv('../../data/final/rca.csv', index=False)
rca

  0%|          | 0/25 [00:00<?, ?it/s]

fMRI_text_hyper_align:   0%|          | 0/292 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

                        norm  train_n   r2_mean     r2_sd check
155         Disgusted_Binder       68  0.062369  0.231635  pass
284         goals_vanarsdall       76  0.045279  0.154537  pass
204       emotionality_brown       86  0.035064  0.020081  pass
178             aoa_stration       20  0.033426  0.307480  pass
63        Humor_Overall_Enge      252  0.029178  0.081281  pass
278               this_rocca       68  0.018550  0.074744  pass
286  concreteness_vanarsdall       76  0.017739  0.111521  pass
144             Human_Binder       68  0.017543  0.155931  pass
109        Complexity_Binder       40  0.006460  0.175994  pass
157         Surprised_Binder       68  0.005446  0.065649  pass


norms_sensorimotor:   0%|          | 0/292 [00:00<?, ?it/s]

...

EEG_text:   0%|          | 0/292 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

                    norm  train_n   r2_mean     r2_sd check
117      Audition_Binder      144  0.033027  0.047981  pass
21             DPoS_Brys     2538  0.001931  0.002302  pass
22             DPoS_VanH     2565  0.001127  0.002543  pass
28   Nmeanings_Wordsmyth       68  0.000935  0.094880  pass
73      Emot_Assoc_Trust     1359 -0.002230  0.007843  pass
72   Emot_Assoc_Surprise     1359 -0.002866  0.009414  pass
70   Emot_Assoc_Positive     1359 -0.003166  0.004962  pass
26       Nsenses_WordNet     2414 -0.004530  0.003279  pass
74         Sem_Diversity     2439 -0.005119  0.008655  pass
92         Naming_RT_ELP     2414 -0.005427  0.006751  pass


LexVec_CommonCrawl:   0%|          | 0/292 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma